In [ ]:
import numpy as np
from tqdm import tqdm
from netCDF4 import Dataset
import os, sys
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.io as sio
from datetime import datetime
sys.path.append('/wk171/handsomedong/after_Meeting/')

from utility import load_data, count, idxFromDatetime
from new_fig import plotLongFig
from preprocess.utils_data_collect.terrain_slope import load_shp, mapping, dotProduct

In [ ]:
# load data
path = os.path.join(os.getcwd(), "repro_data")
fileList = [os.path.join(os.getcwd(), "repro_data", i) for i in sorted(os.listdir(path)) if "avg" in i]
# model decision
names = ['PONI_Atten', 'PONI_Atten_Windv1']
fileList = [fileList[5], fileList[6]]
fileList

In [ ]:
# LOAD AI
data_loader = load_data(*fileList, file_num = len(fileList))
all_data, target, init_t, = data_loader._return
# LOAD TERRAIN
filename = "/bk2/handsomedong/DLRA_database/terrain_slope/GIS_terrain.shp"
df = load_shp(filename)
target_map, latList, lonList = df.getMap("高程")
new_map, new_lat, new_lon = mapping(latList, lonList, target_map, (120,120))

In [ ]:
def blurness(data, k_size=3):
    # Moving Average
    # data shape = [H, W]
    pd = k_size//2
    tmp = np.pad(data, ((pd,pd), (pd,pd)), 'constant')
    tmpp = np.copy(tmp)
    for i in range(pd, tmp.shape[0]-pd):
        for j in range(pd, tmp.shape[1]-pd):
            tmpp[i, j] = tmp[i-pd:i+pd+1, j-pd:j+pd+1].mean()
    return tmpp[pd:-pd, pd:-pd]
ec_filedir = "/bk2/handsomedong/DLRA_database/ERA5_reanalysis"
dot = dotProduct(new_map, ec_filedir)
# if new_map is loaded from "高程", please set is_elevation=True
output1, output2 = dot.forward(is_elevation=True)
b_slope_x = blurness(output1, k_size=5)
b_slope_y = blurness(output2, k_size=5)

# not settings

In [ ]:
from netCDF4 import Dataset
from datetime import datetime
import os

# 3h 累積
orig = all_data[0].sum(axis=2).squeeze()
hetr = all_data[1].sum(axis=2).squeeze()
targ = target.sum(axis=1)

orig_head = []; orig_tail = []
hetr_head = []; hetr_tail = []
targ_head = []; targ_tail = []
for i in tqdm(range(0, len(init_t))):
    dt = init_t[i]
    filepath = os.path.join('/bk2/handsomedong/DLRA_database/ERA5_reanalysis/', str(dt.year), 
                            str(dt.year)+'{:02d}'.format(dt.month), 
                            f'era5_{dt.year}{dt.month:02}{dt.day:02}.nc',
                            )
    data = Dataset(filepath, 'r')
    avg_wind = []
    for key in ['u', 'v']:
        wind = data.variables[key][:] # masked array [24, 20, lat, lon]
        wind[np.where(wind.mask!=0)] = np.nan
        wind = np.nanmean(wind[:, -7], axis=(-1, -2)) # -7 is 850 hpa
        avg_wind.append(wind[dt.hour]) # first u then v
    ans = b_slope_x * avg_wind[0] + b_slope_y * avg_wind[1]
    
    orig_head.extend(orig[i, ans >= 1500])
    orig_tail.extend(orig[i, ans <= -1500])
    hetr_head.extend(hetr[i, ans >= 1500])
    hetr_tail.extend(hetr[i, ans <= -1500])
    targ_head.extend(targ[i, ans >= 1500])
    targ_tail.extend(targ[i, ans <= -1500])

# PDF

In [ ]:
l = np.arange(0, 150, 10)
head_count = []
head_count.append(count(orig_head, threshold=l))
head_count.append(count(hetr_head, threshold=l))

tail_count = []
tail_count.append(count(orig_tail, threshold=l))
tail_count.append(count(hetr_tail, threshold=l))
print(head_count[0], '\n', head_count[1])
print(tail_count[0], '\n', tail_count[1])

In [ ]:
fig, ax = plt.subplots(1, 1, num='result', figsize=(7, 5.6), dpi=200, facecolor='w')
for i in range(len(head_count)):
    plt.plot(l,head_count[i],color="C"+str(i), alpha=0.6)
    plt.fill_between(l, 0, head_count[i], facecolor="C"+str(i), alpha = 0.5)
plt.yscale("log")

# RMSE

In [ ]:
rain_thsh = [0, 10, 30, 50, 80, 100, 130]
ticks_name = ['0~10', '10~30', '30~50', '50~80', '80~100', '100~130', '>= 130']

def cal_itv_rmse(pred:list, target:list, threshold):
    diff_thsh = []
    for i in range(len(threshold)):
        if i == len(threshold)-1:
            mask = np.where(np.array(target)>=threshold[i])
            rmse = np.mean((np.array(pred)[mask]-np.array(target)[mask])**2)**0.5
            diff_thsh.append(rmse)
        else:
            mask = np.where((np.array(target)>=threshold[i]) & (np.array(target)<threshold[i+1]))
            rmse = np.mean((np.array(pred)[mask]-np.array(target)[mask])**2)**0.5
            diff_thsh.append(rmse)
    return diff_thsh

def cal_itv_mape(pred:list, target:list, threshold):
    diff_thsh = []
    for i in range(len(threshold)):
        if i == len(threshold)-1:
            mask = np.where(np.array(target)>=threshold[i])
            rmse = np.mean(np.abs((np.array(pred)[mask]-np.array(target)[mask]) / np.array(target)[mask]))
            diff_thsh.append(rmse)
        else:
            mask = np.where((np.array(target)>=threshold[i]) & (np.array(target)<threshold[i+1]))
            rmse = np.mean(np.abs((np.array(pred)[mask]-np.array(target)[mask]) / np.array(target)[mask]))
            diff_thsh.append(rmse)
    return diff_thsh

ans = []
for x, y in zip([orig_head,hetr_head,orig_tail,hetr_tail], [targ_head,targ_head,targ_tail,targ_tail]):
    ans.append(cal_itv_rmse(x, y, rain_thsh)) # ans shape [4][len(rain_thsh)]

# plot 
fig, ax = plt.subplots(2,1, figsize=(10, 10), dpi=300, facecolor='w')
width = 0.3
ax[0].bar(np.arange(len(rain_thsh)), ans[0], width, color='C0', label='orgi')
ax[0].bar(np.arange(len(rain_thsh)) + width, ans[1], width, color='C3', label='History')
ax[0].set_xticks(np.arange(len(rain_thsh)) + width / 2)
ax[0].set_xticklabels(ticks_name)
ax[0].legend(['PONI_Atten', 'PONI_Atten_Wind'])
ax[0].set_ylabel('threshold (mm)')
ax[0].set_title('RMSE')
ax[0].grid(axis='y', ls='--')
ax[0].set_ylim(0,80)
ax[1].bar(np.arange(len(rain_thsh)), ans[2], width, color='C0', label='orgi')
ax[1].bar(np.arange(len(rain_thsh)) + width, ans[3], width, color='C3', label='History')
ax[1].set_xticks(np.arange(len(rain_thsh)) + width / 2)
ax[1].set_xticklabels(ticks_name)
ax[1].set_xlabel('mm')
ax[1].set_ylabel('threshold (mm)')
ax[1].grid(axis='y', ls='--')
ax[1].set_title('RMSE')
ax[1].set_ylim(0,80)

# case study

In [ ]:
# dt = datetime(2019,9,30,9,50)
# dt = datetime(2021,6,4,3,10)
dt = datetime(2019,12,30,23,0)
# dt = datetime(2018,5,7,16,10)
id = idxFromDatetime(init_t, dt)
t_len = 5

case_target = target.sum(axis=1)
case_origin = all_data[0].sum(axis=2).squeeze()
case_hetero = all_data[1].sum(axis=2).squeeze()

### model axis
mat = sio.loadmat('city_lonlat_region5.mat')
citylon = mat['citylon']
citylat = mat['citylat']
del mat
latStart = 20; latEnd = 27;
lonStart = 118; lonEnd = 123.5;
lat = np.linspace(latStart,latEnd,561)
lon = np.linspace(lonStart,lonEnd,441)
lon, lat = np.meshgrid(lon[215:335], lat[325:445])
terrain_lat = np.linspace(20,27,561)[325:445]
terrain_lon = np.linspace(118,123.5,441)[215:335]

#set colorbar
cwbRR = mpl.colors.ListedColormap(['#FFFFFF', '#9CFCFF', '#03C8FF', '#059BFF', '#0363FF',
                                   '#059902', '#39FF03', '#FFFB03', '#FFC800', '#FF9500',
                                   '#FF0000', '#CC0000', '#990000', '#960099', '#C900CC',
                                   '#FB00FF', '#FDC9FF'])
bounds = [ 0, 1, 2, 5, 10, 15, 20, 30, 40, 50, 70, 90, 110, 130, 150, 200, 300]
norm = mpl.colors.BoundaryNorm(bounds, cwbRR.N)
    
fig, ax = plt.subplots(4, t_len, figsize=(10, 7), dpi=300, facecolor='w')
for i in range(t_len):
    idx = id+i
    dt = init_t[idx]
    filepath = os.path.join('/bk2/handsomedong/DLRA_database/ERA5_reanalysis/', str(dt.year), 
                            str(dt.year)+'{:02d}'.format(dt.month), 
                            f'era5_{dt.year}{dt.month:02}{dt.day:02}.nc',
                            )
    data = Dataset(filepath, 'r')
    avg_wind = []
    for key in ['u', 'v']:
        wind = data.variables[key][:] # masked array [24, 20, lat, lon]
        wind[np.where(wind.mask!=0)] = np.nan
        wind = np.nanmean(wind[:, -7], axis=(-1, -2)) # -7 is 850 hpa
        avg_wind.append(wind[dt.hour]) # first u then v
    ans = b_slope_x * avg_wind[0] + b_slope_y * avg_wind[1]
    
    #
    mask_head = ans >= 500
    tmp1, tmp2 = np.where(mask_head==True)
    
    
    # plot
    ans[ans<0]=0
    ax[0,i].plot(citylon,citylat,'k',linewidth=0.6)
    ax[0,i].axis([120.6875, 122.1875, 24.0625, 25.5625])# whole area [119, 123, 21, 26]
    ax[0,i].set_aspect('equal')
    ax[0,i].imshow(ans, norm=None, aspect='equal', cmap='binary',
                    extent=[terrain_lon[0],terrain_lon[-1],terrain_lat[-1],terrain_lat[0]],)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    
    ax[1,i].plot(citylon,citylat,'k',linewidth=0.6)
    ax[1,i].axis([120.6875, 122.1875, 24.0625, 25.5625])# whole area [119, 123, 21, 26]
    ax[1,i].set_aspect('equal')
    ax[1,i].pcolormesh(lon, lat, case_target[idx], edgecolors='none',shading='auto', norm=norm, cmap=cwbRR)
    ax[1,i].plot(terrain_lon[tmp2], terrain_lat[tmp1], 'k.', markersize=0.4) 
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])
    
    ax[2,i].plot(citylon,citylat,'k',linewidth=0.6)
    ax[2,i].axis([120.6875, 122.1875, 24.0625, 25.5625])# whole area [119, 123, 21, 26]
    ax[2,i].set_aspect('equal')
    ax[2,i].pcolormesh(lon, lat, case_origin[idx], edgecolors='none',shading='auto', norm=norm, cmap=cwbRR)
    ax[2,i].set_xticks([])
    ax[2,i].set_yticks([])
    
    ax[3,i].plot(citylon,citylat,'k',linewidth=0.6)
    ax[3,i].axis([120.6875, 122.1875, 24.0625, 25.5625])# whole area [119, 123, 21, 26]
    ax[3,i].set_aspect('equal')
    ax[3,i].pcolormesh(lon, lat, case_hetero[idx], edgecolors='none',shading='auto', norm=norm, cmap=cwbRR)
    ax[3,i].set_xticks([])
    ax[3,i].set_yticks([])